In [1]:
import torch
import time
from torchvision import datasets, transforms, models  # datsets  , transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
transform4 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=5)], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # MobileNet-specific
])

In [3]:
dataset_mobilenet=datasets.ImageFolder("/kaggle/input/mepco-tropic-leaf/MepcoTropicLeaf-V1/Database", transform=transform4)

In [4]:
indices=list(range(len(dataset_mobilenet))) # dataset ko numerate karne ke liye
split=int(np.floor(0.70*len(dataset_mobilenet)))
validation=int(np.floor(0.60*split))

# agar tmre pass 100 samples hai
# toh split=70(ie 70% of the dataset)-> ee use hoga training and validation ke liye; remaining (30)30% used hoga as test set
# tb validation=42 (ie 60% of the dataset)-> ee use hoga for training ke liye; remaining (28)40% used hoga as Validation set
# toh phir training= 42%; validation= 28%; test=30%

print(f"length of train size : {validation}")
print(f"length of validation size : {split-validation}")
print(f"length of test size : {len(dataset_mobilenet)-split}")

np.random.shuffle(indices) # dataset me randomness laane ke liye

# ab actual splitting
train_indices, validation_indices, test_indices = (
    indices[:validation], # [:5]->0,1,2,3,4
    indices[validation:split],# [1:3]->1,2
    indices[split:],# [2:]-> 2,3,4,5,........
)

train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
test_sampler = SubsetRandomSampler(test_indices)

# print(list(train_indices))
# print(list(validation_indices))
# print(list(test_indices))

# print(list(train_sampler))
# print(list(validation_sampler))
# print(list(test_sampler))


length of train size : 1585
length of validation size : 1058
length of test size : 1134


In [5]:
targets_size = len(dataset_mobilenet.class_to_idx)# finding the total unique classes and storing it
print(targets_size)
print(list(dataset_mobilenet.class_to_idx.keys()))
num_classes_list = list(dataset_mobilenet.class_to_idx.values())# now numerating them
print(num_classes_list)

50
['Asthma Plant.zip', 'Avaram.zip', 'Balloon vine.zip', 'Bellyache bush (Green).zip', 'Benghal dayflower.zip', 'Big Caltrops.zip', 'Black-Honey Shrub.zip', 'Bristly Wild Grape.zip', 'Butterfly Pea.zip', 'Cape Gooseberry.zip', 'Common Wireweed.zip', 'Country Mallow.zip', 'Crown flower.zip', 'Green Chireta.zip', 'Holy Basil.zip', 'Indian CopperLeaf.zip', 'Indian Jujube.zip', 'Indian Sarsaparilla.zip', 'Indian Stinging Nettle.zip', 'Indian Thornapple.zip', 'Indian wormwood.zip', 'Ivy Gourd.zip', 'Kokilaksha.zip', 'Land Caltrops (Bindii).zip', 'Madagascar Periwinkle.zip', 'Madras Pea Pumpkin.zip', 'Malabar Catmint.zip', 'Mexican Mint.zip', 'Mexican Prickly Poppy.zip', 'Mountain Knotgrass.zip', 'Nalta Jute.zip', 'Night blooming Cereus.zip', 'Panicled Foldwing.zip', 'Prickly Chaff Flower.zip', 'Punarnava.zip', 'Purple Fruited Pea Eggplant.zip', 'Purple Tephrosia.zip', 'Rosary Pea.zip', 'Shaggy button weed.zip', 'Small Water Clover.zip', 'Spiderwisp.zip', 'Square Stalked Vine.zip', 'Stinkin

In [6]:
model2 = models.mobilenet_v2(pretrained=True)

# Freeze all layers first
for param in model2.parameters():
    param.requires_grad = False

# Unfreeze last 3 inverted residual blocks (adjust as needed)
for layer in model2.features[-6:]:  # Last 6 layers
    for param in layer.parameters():
        param.requires_grad = True

# Modify classifier
model2.classifier[1] = nn.Linear(model2.classifier[1].in_features, targets_size)
model2.classifier[0] = nn.Dropout(p=0.3)  # Reduce dropout from 0.5 to 0.3

model2.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model2.classifier[1].in_features, 512),  # Hidden layer
    nn.ReLU(),
    nn.Linear(512, targets_size))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 112MB/s] 


In [7]:
n_features = model2.classifier[1].in_features #mobilenet; number of input features in the first fully connected layer
n_features

1280

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) #checking if GPU is available
model2.to(device)

cuda


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [9]:
criterion = nn.CrossEntropyLoss()  # this include softmax + cross entropy loss
# calculates the loss during training, which will be later used by backpropagation to imporove the models accuracy
optimizer = torch.optim.Adam([
    {'params': model2.features[-6:].parameters(), 'lr': 1e-4},  # Higher LR for unfrozen
    {'params': model2.classifier.parameters(), 'lr': 1e-3}       # Even higher for classifier
], weight_decay=1e-5)  # Reduce weight decay #Add to the optimizer to penalize large weights
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
#adam optimiser is used to optimise the models parameters(weights of the model) to minimise the loss and hence increase the accuracy

In [10]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    dataset_mobilenet, batch_size=batch_size, sampler=train_sampler
)
validation_loader = torch.utils.data.DataLoader(
    dataset_mobilenet, batch_size=batch_size, sampler=validation_sampler
)
test_loader = torch.utils.data.DataLoader(
    dataset_mobilenet, batch_size=batch_size, sampler=test_sampler
)

In [11]:
def plot_losses(train_losses, test_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss', color='blue', marker='o')
    plt.plot(test_losses, label='Validation Loss', color='red', marker='x')
    plt.title('Training and Validation Losses', fontsize=16)
    plt.xlabel('Epochs', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [12]:
def batch_gd_new(model2, criterion, train_loader, test_loader, epochs):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    
    # Memory tracking initialization
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        mem_before = torch.cuda.memory_allocated() / 1024**2  # MB
    
    # Track average inference time
    total_inference_time = 0
    inference_count = 0
    
    for e in range(epochs):
        t0 = datetime.now()
        train_loss = []
        
        # Training phase
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Measure inference time
            start_time = time.time()
            optimizer.zero_grad()
            output = model2(inputs)
            inference_time = time.time() - start_time
            
            total_inference_time += inference_time
            inference_count += 1
            
            loss = criterion(output, targets)
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()

        train_loss = np.mean(train_loss)
        test_loss = []
        
        # Validation phase
        model2.eval()
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                output = model2(inputs)
                loss = criterion(output, targets)
                test_loss.append(loss.item())
        model2.train()

        test_loss = np.mean(test_loss)
        train_losses[e] = train_loss
        test_losses[e] = test_loss
        
        dt = datetime.now() - t0
        print(f"Epoch {e+1}/{epochs} Train loss: {train_loss:.4f} Val loss: {test_loss:.4f} Duration: {dt}")

    # Memory and timing results
    avg_inference_time = total_inference_time / inference_count if inference_count > 0 else 0
    print(f"\nTraining Summary:")
    print(f"Average inference time per batch: {avg_inference_time:.4f} seconds")
    
    if torch.cuda.is_available():
        mem_after = torch.cuda.memory_allocated() / 1024**2
        mem_used = mem_after - mem_before
        print(f"GPU Memory used: {mem_used:.2f} MB")
    
    return train_losses, test_losses

In [14]:
train_losses, test_losses= batch_gd_new(model2, criterion, train_loader, validation_loader, 10)
plot_losses(train_losses, test_losses)

Epoch 1/10 Train loss: 3.1770 Val loss: 1.9959 Duration: 0:01:20.495977
Epoch 2/10 Train loss: 1.2289 Val loss: 0.7246 Duration: 0:01:01.400397
Epoch 3/10 Train loss: 0.5353 Val loss: 0.4285 Duration: 0:01:01.529139
Epoch 4/10 Train loss: 0.3124 Val loss: 0.3449 Duration: 0:01:00.286682
Epoch 5/10 Train loss: 0.2576 Val loss: 0.2943 Duration: 0:01:00.150772
Epoch 6/10 Train loss: 0.1985 Val loss: 0.2663 Duration: 0:00:59.996434
Epoch 7/10 Train loss: 0.1574 Val loss: 0.2392 Duration: 0:00:59.868397
Epoch 8/10 Train loss: 0.1408 Val loss: 0.2623 Duration: 0:00:59.766058
Epoch 9/10 Train loss: 0.1204 Val loss: 0.2051 Duration: 0:00:59.978305
Epoch 10/10 Train loss: 0.0741 Val loss: 0.2375 Duration: 0:00:59.771830

Training Summary:
Average inference time per batch: 0.0117 seconds
GPU Memory used: 64.31 MB


In [15]:
def accuracy(loader):
    n_correct = 0
    n_total = 0
    model2.cuda()
    for inputs, targets in loader:
        inputs, targets = inputs.cuda(), targets.cuda()
        outputs = model2(inputs)
        #print(outputs)
        _, predictions = torch.max(outputs, 1)
        n_correct += (predictions == targets).sum().item()
        n_total += targets.shape[0]

    acc = n_correct / n_total
    return acc

In [16]:
train_acc = accuracy(train_loader)
test_acc = accuracy(test_loader)
validation_acc = accuracy(validation_loader)

print(
    f"Train Accuracy : {train_acc}\nTest Accuracy : {test_acc}\nValidation Accuracy : {validation_acc}"
)

Train Accuracy : 0.9753943217665615
Test Accuracy : 0.9250440917107584
Validation Accuracy : 0.9177693761814745
